# Retrieval models
This notebook aims to implement the classical retrieval models based on tokenized texts and a defined word corpus

In [1]:
import pandas as pd
import pickle
import numpy as np
import math
from abc import ABC, abstractmethod
from sklearn import metrics

## Dataset import

In [2]:
word_corpus = pd.read_csv("../datasets/20news-word-corpus-2k.csv")
test_filter = np.load("../datasets/test_filter.npy")
classes = np.load("../datasets/20-news-classes.npy")
with open("../datasets/20-news-processed-no-singles.pickle", "rb") as f:
    dataset = pickle.load(f)
len(word_corpus)

2423

## Evaluation class

A class to compute P@10, P@20, P@50, P@100 e MAP.

In [3]:
class RetrievalEvaluation:
    
    def __init__(self, ranked_lists:np.ndarray, classes:np.ndarray):
        self.ranked_lists = ranked_lists
        self.classes = classes
        self.class_size = 1000
        self.n = len(ranked_lists)
        
    def p_at_n(self, n:int)->float:
        p_total = 0
        for rank in self.ranked_lists:
            p = 0
            target_class = self.classes[rank[0]]
            for i in range(n):
                if self.classes[rank[i]] == target_class:
                    p += 1
            p = p / n
            p_total += p
        return p_total / self.n
    
    def p_at_10(self)->float:
        return self.p_at_n(n=10)
    
    def p_at_20(self)->float:
        return self.p_at_n(n=20)
    
    def p_at_50(self)->float:
        return self.p_at_n(n=50)
    
    def p_at_100(self)->float:
        return self.p_at_n(n=100)
    
    def computeAveragePrecision(self, rk, d=1000):
        sumrj = 0
        curPrecision = 0
        sumPrecision = 0
        qClass = self.classes[rk[0]]
        for i in range(d):
            imgi = rk[i]
            imgiClass = self.classes[imgi]
            if (qClass == imgiClass):
                sumrj = sumrj + 1
                posi = i + 1
                curPrecision = sumrj / posi
                sumPrecision += curPrecision
        nRel = self.class_size
        l = len(rk)
        avgPrecision = sumPrecision / min(l, nRel)
        return avgPrecision

    def compute_map(self):
        acumAP = 0
        for rk in self.ranked_lists:
            acumAP += self.computeAveragePrecision(rk)
        return acumAP / self.n
    
    def evaluate_all(self) -> None:
        print("=========== Evaluation Procedure ===========")
        print("Evaluation dataset size:", self.n)
        print("Precision at 10:", self.p_at_10())
        print("Precision at 20:", self.p_at_20())
        print("Precision at 50:", self.p_at_50())
        print("Precision at 100:", self.p_at_100())
        print("Map:", self.compute_map())

## The Retrieval model abstract class

In order to facilitate the implementation of new models, we first create an AbstractClass to define the expected behavior of the models

In [6]:
class VectorRetrievalModel(ABC):
 
    def __init__(self, word_corpus: pd.DataFrame):
        self.word_corpus = word_corpus.word.to_list()
        # self.word_corpus.sort()
        self.dataset = None
        self.metric = None
        super().__init__()
    
    def get_dataset(self) -> np.ndarray:
        return self.dataset
    
    @abstractmethod
    def convert_dataset(self, dataset:list):
        pass
    
    @abstractmethod
    def convert_item(self, item:list) -> list:
        pass
    
    def compute_ranked_lists(self, metric:str = None, test_filter=None) -> np.ndarray:
        if self.dataset is None:
            raise Exception("No dataset has been defined, please create a new one with convert_dataset function.")
        
        if self.metric is None:
            raise Exception("No metric defined for the model.")
        
        if metric is None:
            metric = self.metric
            
        target_ranked_lists = self.dataset if test_filter is None else self.dataset[test_filter]
            
        
        ranked_lists = []

        if metric == "cosine":
            distances = metrics.pairwise.cosine_distances(target_ranked_lists, self.dataset)
        else:
            distances = metrics.pairwise.euclidean_distances(target_ranked_lists, self.dataset)
        
        for item in distances:
            rank_map = np.argsort(item)
            ranked_lists.append(rank_map)
        self.ranked_lists = np.asarray(ranked_lists)
    
    def get_ranked_lists(self) -> np.ndarray:
        return self.ranked_lists

## Binary model
A vetor with values 1 or 0 for each word of the word corpus, representing if they are in the encoded text tokens.

In [7]:
class BinaryModel(VectorRetrievalModel):
    
    def __init__(self, word_corpus:pd.DataFrame):
        super().__init__(word_corpus)
        self.metric = 'cosine'
        
    def convert_dataset(self, dataset:list):
        binary_dataset = []
        for item in dataset:
            binary_dataset.append(self.convert_item(item))
        self.dataset = np.asarray(binary_dataset)
    
    def convert_item(self, item:list)-> list:
        binary_item = []
        for word in self.word_corpus:
            value = 1 if word in item else 0
            binary_item.append(value)
        return binary_item

In [8]:
binary_model = BinaryModel(word_corpus)
binary_model.convert_dataset(dataset)
binary_dataset = binary_model.get_dataset()
binary_dataset.shape

(19997, 2423)

In [9]:
binary_ranked_list = binary_model.compute_ranked_lists()

## Evaluation test

In [ ]:
train = pd.read_csv("../datasets/20news-train-filtered.csv")
test = pd.read_csv("../datasets/20news-test.csv")

In [ ]:
len(test)

2000

In [13]:
classes = train.category_id.to_list()
test_ids = test.id.to_list()

In [90]:
classes[12000]

12

In [49]:
test_classes = test.category_id.to_list()

In [74]:
len(train.id.unique())

19997

In [16]:
test_filter = train.id.isin(test_ids).to_list()

In [76]:
test_ranked_lists = binary_ranked_list[test_filter]

In [77]:
test_ranked_lists.shape

(2000, 19997)

In [87]:
rk = test_ranked_lists[0]

In [85]:
evaluation = RetrievalEvaluation(np.asarray([test_ranked_lists[0]]), classes)

In [86]:
evaluation.p_at_10()

0.1

In [91]:
rk = test_ranked_lists[0]
for i in range(10):
    print(rk[i], "->", classes[rk[i]])

4 -> 0
1354 -> 1
5303 -> 5
13131 -> 13
17633 -> 17
8882 -> 8
17488 -> 17
17229 -> 17
14280 -> 14
17570 -> 17


## Bag-of-Words

The bag of words model is an expansion of the binary model, where the text is represented by a vector containing the count of each word from the word corpus in the converted text.

In [98]:
class BagOfWordsModel(VectorRetrievalModel):
    
    def __init__(self, word_corpus:pd.DataFrame):
        super().__init__(word_corpus)
        self.metric = "cosine"
        
    def convert_dataset(self, dataset:list):
        bow_dataset = []
        for item in dataset:
            bow_dataset.append(self.convert_item(item))
        self.dataset = np.asarray(bow_dataset)
    
    def convert_item(self, item:list)-> list:
        bow_item = []
        for word in self.word_corpus:
            bow_item.append(item.count(word))
        return bow_item

In [28]:
bow_model = BagOfWordsModel(word_corpus)
bow_dataset = bow_model.convert_dataset(dataset)
bow_dataset.shape

(19997, 1289)

In [29]:
np.save("../datasets/20-news-bow-model", bow_dataset)

## TF-IDF model
The TF-IDF model computes values for each word based on its ocurrence in each text, in the full corpus and in how many texts it appears.

In [6]:
class TfIdfModel(VectorRetrievalModel):
    
    def __init__(self, word_corpus:pd.DataFrame):
        super().__init__(word_corpus)
        self.metric = "cosine"
    
    def compute_idf(self, dataset:list):
        word_idf = []
        dataset_size = len(dataset)
        for index, word in enumerate(self.word_corpus):
            word_idf.append(0)
            for item in dataset:
                if word in item:
                    word_idf[index] += 1
            if word_idf[index] == 0:
                continue
            word_idf[index] = math.log(dataset_size/word_idf[index], 2)
        self.word_idf = np.asarray(word_idf)
        
    def convert_dataset(self, dataset:list):
        # Calcular o idf para cada palavra.
        self.compute_idf(dataset)
        
        tf_idf_dataset = []
        for item in dataset:
            tf_idf_dataset.append(self.convert_item(item))
        self.dataset = np.asarray(tf_idf_dataset)
    
    def compute_tf(self, word:str, item:list):
        word_count = item.count(word)
        if word_count == 0:
            return 0
        else:
            return 1 + math.log(word_count, 2)
    
    def convert_item(self, item:list)-> list:
        tf_idf_item = []
        for index, word in enumerate(self.word_corpus):
            tf = self.compute_tf(word, item)
            tf_idf = tf * self.word_idf[index]
            tf_idf_item.append(tf_idf)
        return tf_idf_item

In [11]:
np.save("../datasets/test_filter", test_filter)

In [8]:
tf_idf_model = TfIdfModel(word_corpus)
tf_idf_model.convert_dataset(dataset)
tf_idf_dataset = tf_idf_model.get_dataset()
tf_idf_dataset.shape

(19997, 2423)

In [9]:
tf_idf_model.compute_ranked_lists(metric="cosine", test_filter=test_filter)

In [10]:
ranked_lists = tf_idf_model.get_ranked_lists()
print(ranked_lists.shape)
evaluation = RetrievalEvaluation(ranked_lists, classes)
evaluation.evaluate_all()

(2000, 19997)
=========== Evaluation Procedure ===========
Evaluation dataset size: 2000
Precision at 10: 0.6672999999999998
Precision at 20: 0.5894750000000006
Precision at 50: 0.5045000000000003
Precision at 100: 0.44692000000000015
Map: 0.11126248061136677


In [23]:
rk = ranked_lists[999]
for i in range(10):
    print(rk[i], "->", classes[rk[i]])

9987 -> 9
9859 -> 9
16783 -> 16
9651 -> 9
9368 -> 9
10806 -> 10
9315 -> 9
10233 -> 10
10214 -> 10
10991 -> 10


In [76]:
np.save("../datasets/20-news-tf-idf-model", tf_idf_dataset)

In [34]:
test_dataset = tf_idf_model.get_dataset()
test_a = test_dataset[0:10]
distances = metrics.pairwise.cosine_distances(test_a, test_dataset)
distances.shape

(10, 19997)

## BM Models

BM models are probalilistic models that take into account the size of each text, alongside other elements already demonstrated other elements explored in the models implemented above.

Probabilitic models originally do not create a vector representation to each text in the corpus. They work as a direct computation between a query text and all the other texts in the corpus, retrieving a similarity score for each compared pair.

In this implementation, the equation is computed for each word in the corpus, resulting in a vector with the value that would be sum in the original proposition.

### BM1

BM1 only takes into account the relation between all documents and in how many documents each term appears. 

In [14]:
class BM1Model(VectorRetrievalModel):
    
    def __init__(self, word_corpus:pd.DataFrame):
        super().__init__(word_corpus)
        self.metric = "cosine"
    
    def compute_idf(self, dataset:list):
        word_idf = []
        dataset_size = len(dataset)
        for index, word in enumerate(self.word_corpus):
            word_idf.append(0)
            for item in dataset:
                if word in item:
                    word_idf[index] += 1
            word_idf[index] = math.log((dataset_size - word_idf[index] + 0.5) / (word_idf[index] + 0.5), 2)
        self.word_idf = np.asarray(word_idf)
        
    def convert_dataset(self, dataset:list):
        # Calcular o idf para cada palavra.
        self.compute_idf(dataset)
        
        tf_idf_dataset = []
        for item in dataset:
            tf_idf_dataset.append(self.convert_item(item))
        self.dataset = np.asarray(tf_idf_dataset)
    
    def compute_tf(self, word:str, item:list):
        word_count = item.count(word)
        if word_count == 0:
            return 0
        else:
            return 1 + math.log(word_count, 2)
    
    def convert_item(self, item:list)-> list:
        tf_idf_item = []
        for index, word in enumerate(self.word_corpus):
            tf_idf = self.word_idf[index] if item.count(word) > 0 else 0
            tf_idf_item.append(tf_idf)
        return tf_idf_item

In [ ]:
bm1_model = BM1Model(word_corpus)
bm1_model.convert_dataset(dataset)
bm1_dataset = bm1_model.get_dataset()
bm1_dataset.shape

In [16]:
bm1_dataset[0]

array([1.06394614, 1.76245565, 1.13712578, ..., 0.        , 0.        ,
       0.        ])

### BM11

BM11 Applies a term-factor that also takes into account the average size of the documents in the corpus, alonside with the size of the current document

In [11]:
class BM11Model(VectorRetrievalModel):
    
    def __init__(self, word_corpus:pd.DataFrame, k1 = 1.0):
        super().__init__(word_corpus)
        self.metric = "cosine"
        self.k1 = k1
    
    def compute_idf(self, dataset:list):
        word_idf = []
        dataset_size = len(dataset)
        for index, word in enumerate(self.word_corpus):
            word_idf.append(0)
            for item in dataset:
                if word in item:
                    word_idf[index] += 1
            word_idf[index] = math.log((dataset_size - word_idf[index] + 0.5) / (word_idf[index] + 0.5), 2)
        self.word_idf = np.asarray(word_idf)
        
    def compute_average_size(self, dataset:list):
        total_size = 0
        for item in dataset:
            total_size += len(item)
        self.average_size = total_size / len(dataset)
    
    def convert_dataset(self, dataset:list):
        # Calcular o idf para cada palavra.
        self.compute_idf(dataset)
        self.compute_average_size(dataset)
        
        tf_idf_dataset = []
        for item in dataset:
            tf_idf_dataset.append(self.convert_item(item))
        self.dataset = np.asarray(tf_idf_dataset)
    
    def compute_tf(self, word:str, item:list, item_size:int):
        word_count = item.count(word)
        if word_count == 0:
            return 0
        else:
            return ((self.k1 + 1) * word_count) / ((self.k1 * item_size / self.average_size) + word_count) 
    
    def convert_item(self, item:list)-> list:
        tf_idf_item = []
        item_size = len(item)
        for index, word in enumerate(self.word_corpus):
            tf = self.compute_tf(word, item, item_size)
            tf_idf = tf * self.word_idf[index] if item.count(word) > 0 else 0
            tf_idf_item.append(tf_idf)
        return tf_idf_item

In [ ]:
bm11_model = BM11Model(word_corpus)
bm11_model.convert_dataset(dataset)
bm11_dataset = bm11_model.get_dataset()
bm11_dataset.shape

In [13]:
bm11_dataset[0]

array([0.26026642, 1.60517239, 0.27816789, ..., 0.        , 0.        ,
       0.        ])

### BM15

BM15 removes BM11's analysis over the lenght of the documents, only using constant K1 in the tf statement.

In [14]:
class BM15Model(VectorRetrievalModel):
    
    def __init__(self, word_corpus:pd.DataFrame, k1 = 1.0):
        super().__init__(word_corpus)
        self.metric = "cosine"
        self.k1 = k1
    
    def compute_idf(self, dataset:list):
        word_idf = []
        dataset_size = len(dataset)
        for index, word in enumerate(self.word_corpus):
            word_idf.append(0)
            for item in dataset:
                if word in item:
                    word_idf[index] += 1
            word_idf[index] = math.log((dataset_size - word_idf[index] + 0.5) / (word_idf[index] + 0.5), 2)
        self.word_idf = np.asarray(word_idf)
        
    def compute_average_size(self, dataset:list):
        total_size = 0
        for item in dataset:
            total_size += len(item)
        self.average_size = total_size / len(dataset)
    
    def convert_dataset(self, dataset:list):
        # Calcular o idf para cada palavra.
        self.compute_idf(dataset)
        self.compute_average_size(dataset)
        
        tf_idf_dataset = []
        for item in dataset:
            tf_idf_dataset.append(self.convert_item(item))
        self.dataset = np.asarray(tf_idf_dataset)
    
    def compute_tf(self, word:str, item:list):
        word_count = item.count(word)
        if word_count == 0:
            return 0
        else:
            return ((self.k1 + 1) * word_count) / (self.k1 + word_count) 
    
    def convert_item(self, item:list)-> list:
        tf_idf_item = []
        for index, word in enumerate(self.word_corpus):
            tf = self.compute_tf(word, item)
            tf_idf = tf * self.word_idf[index] if item.count(word) > 0 else 0
            tf_idf_item.append(tf_idf)
        return tf_idf_item

In [15]:
bm15_model = BM15Model(word_corpus)
bm15_model.convert_dataset(dataset)
bm15_dataset = bm15_model.get_dataset()
bm15_dataset.shape

(19997, 1289)

In [16]:
bm15_dataset[0]

array([1.06394614, 3.02135255, 1.13712578, ..., 0.        , 0.        ,
       0.        ])

### BM25

BM25 is the combination of BM11 and BM15 based on a b (beta) constant.

In [22]:
class BM25Model(VectorRetrievalModel):
    
    def __init__(self, word_corpus:pd.DataFrame, k = 1.2, b = 0.8):
        super().__init__(word_corpus)
        self.metric = "cosine"
        self.k = k
        self.b = b
    
    def compute_idf(self, dataset:list):
        word_idf = []
        dataset_size = len(dataset)
        for index, word in enumerate(self.word_corpus):
            word_idf.append(0)
            for item in dataset:
                if word in item:
                    word_idf[index] += 1
            word_idf[index] = math.log((dataset_size - word_idf[index] + 0.5) / (word_idf[index] + 0.5), 2)
        self.word_idf = np.asarray(word_idf)
        
    def compute_average_size(self, dataset:list):
        total_size = 0
        for item in dataset:
            total_size += len(item)
        self.average_size = total_size / len(dataset)
    
    def convert_dataset(self, dataset:list):
        # Calcular o idf para cada palavra.
        self.compute_idf(dataset)
        self.compute_average_size(dataset)
        
        tf_idf_dataset = []
        for item in dataset:
            tf_idf_dataset.append(self.convert_item(item))
        self.dataset = np.asarray(tf_idf_dataset)
    
    def compute_tf(self, word:str, item:list, item_size:int):
        word_count = item.count(word)
        if word_count == 0:
            return 0
        else:
            upper = (self.k + 1) * word_count
            bottom = (self.k * (1 - self.b)) + (self.k * self.b * self.average_size / item_size) + word_count
            return upper / bottom 
    
    def convert_item(self, item:list)-> list:
        tf_idf_item = []
        item_size = len(item)
        for index, word in enumerate(self.word_corpus):
            tf = self.compute_tf(word, item, item_size)
            tf_idf = tf * self.word_idf[index] if item.count(word) > 0 else 0
            tf_idf_item.append(tf_idf)
        return tf_idf_item

In [23]:
bm25_model = BM25Model(word_corpus)
bm25_model.convert_dataset(dataset)
bm25_dataset = bm25_model.get_dataset()
bm25_dataset.shape

(19997, 1289)

In [19]:
bm25_dataset[0]

array([1.57095946, 3.32825701, 1.67901216, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
train = pd.read_csv("../datasets/20news-train-filtered.csv")
test = pd.read_csv("../datasets/20news-test.csv")
classes = train.category_id.to_list()
test_ids = test.id.to_list()
test_filter = train.id.isin(test_ids).to_list()

In [24]:
bm25_model.compute_ranked_lists(metric="cosine", test_filter=test_filter)
ranked_lists = bm25_model.get_ranked_lists()
print(ranked_lists.shape)
evaluation = RetrievalEvaluation(ranked_lists, classes)
evaluation.evaluate_all()

(2000, 19997)
=========== Evaluation Procedure ===========
Evaluation dataset size: 2000
Precision at 10: 0.6366999999999985
Precision at 20: 0.5548249999999999
Precision at 50: 0.4652
Precision at 100: 0.41125000000000095
Map: 0.09621196917989455
